# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 32 new papers today
          14 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/13 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.11987


extracting tarball to tmp_2212.11987...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.11990


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.11987/aanda.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.12314


extracting tarball to tmp_2212.12314...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.12331


/tmp/ipykernel_2088/4030337529.py:34: LatexWarning: 2212.12314 did not run properly
'utf-8' codec can't decode byte 0xa8 in position 23696: invalid start byte
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


extracting tarball to tmp_2212.12331... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11991-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11991) | **Vertical motion in the Galactic disc: unwinding the Snail**  |
|| <mark>Neige Frankel</mark>, Jo Bovy, Scott Tremaine, David W. Hogg |
|*Appeared on*| *2022-12-26*|
|*Comments*| *9 pages, 8 figures. Submitted to MNRAS (constructive comments are welcome)*|
|**Abstract**| The distribution of stars in the Milky Way disc shows a spiral structure--the Snail--in the space of velocity and position normal to the Galactic mid-plane. The Snail appears as straight lines in the vertical frequency--vertical phase plane when effects from sample selection are removed. Their slope has the dimension of inverse time, with the simplest interpretation being the inverse age of the Snail. Here, we devise and fit a simple model in which the spiral starts as a lopsided perturbation from steady state, that winds up into the present-day morphology. The winding occurs because the vertical frequency decreases with vertical action. We use data from stars in Gaia EDR3 that have measured radial velocities, pruned by simple distance and photometric selection functions. We divide the data into boxels of dynamical invariants (radial action, angular momentum); our model fits the data well in many of the boxels. The model parameters have physical interpretations: one, $A$, is a perturbation amplitude, and one, $t$, is interpretable in the simplest models as the time since the event that caused the Snail. We find trends relating the strength and age to angular momentum: (i) the amplitude $A$ is small at low angular momentum ($<1\,600\mathrm{\,kpc\,km\,s}^{-1}$ or guiding-centre radius $< 7.3\,$kpc), and over a factor of three larger, with strong variations, in the outer disc; (ii) there is no single well-defined perturbation time, with $t$ varying between 0.2 and 0.6 Gyr. Residuals between the data and the model display systematic trends, implying that the data call for more complex models. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.12039-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.12039) | **PHANGS-JWST First Results: Massive Young Star Clusters and New Insights  from JWST Observations of NGC 1365**  |
|| Bradley C. Whitmore, et al. -- incl., <mark>Janice C. Lee</mark>, <mark>Kiana F. Henny</mark>, <mark>Stephen Hannon</mark>, <mark>Jonathan D. Henshaw</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2022-12-26*|
|*Comments*| *17 pages, 9 figures, accepted for publications as part of PHANGS-JWST ApJL Focus Issue*|
|**Abstract**| A primary new capability of JWST is the ability to penetrate the dust in star forming galaxies to identify and study the properties of young star clusters that remain embedded in dust and gas. In this paper we combine new infrared images taken with JWST with our optical HST images of the star-bursting barred (Seyfert2) spiral galaxy NGC 1365. We find that this galaxy has the richest population of massive young clusters of any known galaxy within 30 Mpc, with $\sim$ 30 star clusters that are more massive than 10$^6$ Msolar and younger than 10 Myr. Sixteen of these clusters are newly discovered from our JWST observations. An examination of the optical images reveals that 4 of 30 ($\sim$13$\%$) are so deeply embedded that they cannot be seen in the I band (AV $\gt$ 10 mag), and that 11 of 30 ($\sim$37$\%$) are missing in the HST B band, so age and mass estimates from optical measurements alone are challenging. These numbers suggest that massive clusters in NGC 1365 remain obscured in the visible for $\sim$ 1.3 $\pm$ 0.7 Myr, and are either completely or partially obscured for $\sim$ 3.7 $\pm$ 1.1 Myr. We also use the JWST observations to gain new insights into the triggering of star cluster formation by the collision of gas and dust streamers with gas and dust in the bar. The JWST images reveal previously unknown structures (e.g., bridges and overshoot regions from stars that form in the bar) that help us better understand the orbital dynamics of barred galaxies and associated star-forming rings. Finally, we note that the excellent spatial resolution of the NIRCAM F200W filter provides a better way to separate barely resolved compact clusters from individual stars based on their sizes. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11987-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11987) | **The phase spiral in Gaia DR3**  |
|| T. Antoja, et al. -- incl., <mark>P. Ramos</mark> |
|*Appeared on*| *2022-12-26*|
|*Comments*| *submitted to A&A*|
|**Abstract**| We analysed the phase spiral with data from Gaia DR3. We used an edge detection algorithm to find the border of the phase spiral, allowing us to robustly quantify its shape at different positions. We calculated the time of onset of the phase-mixing by determining the different turns of the phase spiral and using the vertical frequencies from standard potential models of the MW. We find that the phase spiral extends down to -1.2 kpc in height below the plane (about 3 to 5 scale heights of the thin disc) and beyond $\pm 50$ km/s in $V_Z$. We see a secondary branch mostly at positive vertical velocities in the local phase spiral when coloured by azimuthal velocity, and also at different angular momentum in the counts projection. We also find complex variations of the phase spirals with angular momentum and azimuth. All these possibly provide evidence of multiple perturbations (from different times or from different perturbers) and/or that the disc is affected by complex phase mixing processes. We detect the phase spiral from 6 to 11 kpc and find signatures of vertical asymmetries 1-2 kpc beyond this range. We measure small but clear variations with azimuth. When we determine the phase mixing times from the phase spiral at different angular momenta and using the different spiral turns (at different $Z$) we obtain inconsistent times with systematic differences (times increasing with $|L_Z|$ and with $|Z|$). Our determinations are mostly in the range of [0.3-0.9] Gyr, with an average of 0.5 Gyr. The inconsistencies do not change when using different potential models for the MW, different stellar distances and frequencies for different kinetic temperatures. They could stem from the inconsistency of potential models with the true MW, and from too simple modelling, in particular neglecting self-gravity, not considering the multiple perturbations and the interference with other processes. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.12331-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.12331) | **The optical spectral features of 27 Fermi blazars**  |
|| <mark>Bing-Kai Zhang</mark>, et al. |
|*Appeared on*| *2022-12-26*|
|*Comments*| *9 pages, 14 figures, accepted by MNRAS*|
|**Abstract**| A spectral variation accompanied with flux variability is a commonly-observed phenomenon for blazars. In order to further investigate the optical spectral feature of blazars, we have collected the long-term optical V and R band data of 27 blazars (14 BL Lacs and 13 FSRQs), and calculated their optical spectral indices. The results show that the spectral indices vary with respect to the brightness for all of these blazars. In general, the optical spectrum progressively becomes flatter (or steeper), when the brightness increases. However the spectrum changes more and more slowly, until it tends to be stable. In other words, the source becomes bluer (or redder) and then gradually stabilizes when it brightens, which are briefly named the bluer-stable-when-brighter (BSWB) and redder-stable-when-brighter (RSWB) behaviors, respectively. Thirteen of the 14 BL Lacs show the BSWB behavior, with an exception of AO 0235+164. On the contrary, most of FSRQs (10 out of 13) exhibit the RSWB trend. It is confirmed that blazars follow the two universal optical spectral behaviors, namely, BSWB and RSWB. The model of two constant-spectral-index components can well explain the optical spectral features qualitatively and quantitatively. The results illustrate that the optical emission are mainly composed of two stable-color components, i.e., less variable thermal emission and high variable synchrotron radiation. And in most cases, the thermal component of FSRQs is redder than that of synchrotron radiation, whereas BL Lacs are the opposite. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.12087-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.12087) | **TESS observations of the Pleiades cluster: a nursery for delta Scuti  stars**  |
|| Timothy R. Bedding, et al. -- incl., <mark>Daniel R. Hey</mark>, <mark>Daniel Huber</mark>, <mark>Yaguang Li</mark>, <mark>Guillermo Torres</mark> |
|*Appeared on*| *2022-12-26*|
|*Comments*| *submitted to AAS journals*|
|**Abstract**| We studied 89 A- and F-type members of the Pleiades open cluster, including five escaped members. We measured projected rotational velocities (v sin i) for 49 stars and confirmed that stellar rotation causes a broadening of the main sequence in the color-magnitude diagram. Using time-series photometry from NASA's TESS Mission (plus one star observed by Kepler/K2), we detected delta Scuti pulsations in 36 stars. The fraction of Pleiades stars in the middle of the instability strip that pulsate is unusually high (over 80%), and their range of effective temperatures agrees well with theoretical models. On the other hand, the characteristics of the pulsation spectra are varied and do not correlate with stellar temperature, calling into question the existence of a useful nu_max relation for delta Scutis, at least for young stars. By including delta Scuti stars observed in the Kepler field, we show that the instability strip is shifted to the red with increasing distance by interstellar reddening. Overall, this work demonstrates the power of combining observations with Gaia and TESS for studying pulsating stars in open clusters. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.12314-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.12314) | **The first flare observation with a new solar microwave spectrometer  working in 35-40 GHz**  |
|| F. Yan, et al. -- incl., <mark>L. Zhang</mark> |
|*Appeared on*| *2022-12-26*|
|*Comments*| *17 pages, 5 figures*|
|**Abstract**| The microwave spectrum contains valuable information about solar flares. Yet, the present spectral coverage is far from complete and broad data gaps exist above 20 GHz. Here we report the first flare (the X2.2 flare on 2022 April 20) observation of the newly-built Chashan Broadband Solar millimeter spectrometer (CBS) working from 35 to 40 GHz. We use the CBS data of the new Moon to calibrate,and the simultaneous NoRP data at 35 GHz to cross-calibrate. The impulsive stage has three local peaks with the middle one being the strongest and the maximum flux density reaches 9300 SFU at 35-40 GHz. The spectral index of the CBS data (alpha_C) for the major peak is mostly positive, indicating the gyrosynchrotron turnover frequency (nu_t) goes beyond 35-40 GHz. The frequency nu_t is smaller yet still larger than 20 GHz for most time of the other two peaks according to the spectral fittings with NoRP-CBS data. The CBS index manifests the general rapid-hardening-then-softening trend for each peak and gradual hardening during the decay stage, agreeing with the fitted optically-thin spectral index (alpha_tn) for nu_t < 35 GHz. In addition, the obtained turnover frequency during the whole impulsive stage correlates well with the corresponding intensity (I_t) according to a power-law dependence (It~nu_t^4.8) with a correlation coefficient of 0.82.This agrees with earlier studies on flares with low turnover frequency (<17 GHz), yet being reported for the first time for events with a high turnover frequency (>20 GHz). |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xa8 in position 23696: invalid start byte</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.12093-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.12093) | **The environments of Type Ia supernovae with different relative  equivalent width of Si II feature in their spectra**  |
|| Xiang-Cun Meng, et al. -- incl., <mark>Ju-Jia Zhang</mark>, <mark>Xu-Lin Zhao</mark>, <mark>Li-Ping Li</mark> |
|*Appeared on*| *2022-12-26*|
|*Comments*| *17 pages, 13 figures, accepted for publication in ApJ*|
|**Abstract**| Although type Ia supernovae are so important in many astrophysical field, e.g. in cosmology, their explosion mechanism and progenitor system are still unclear. In physics, the relative equivalent width (REW) of the Si II 635.5 nm absorption feature reflects the velocity interval of silicon in the supernova ejecta and then may provide constraints on the explosion mechanism of SNe Ia. In this paper, we divide the SNe Ia into broad line (BL) and normal line (NL) subsamples based on their REW of Si II 635.5 nm absorption lines around maximum light, and find that the BL SNe Ia have a dimmer mean brightness than NL ones, which possibly results from their different metallicities. However, based on the pixel statistics study on the environments of two subsamples, we do not find any significant potential difference on the environments between BL and NL SNe Ia, which implies that the explosion mechanism of SNe Ia could be independent of their progenitor populations. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xb3 in position 67131: invalid start byte</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11992-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11992) | **Design of Chemical Propellant Thruster to Deorbit Nano satellite:  StudSat II**  |
|| Prabin Sherpaili, Roshan Sah, <mark>Sandesh Hegde</mark>, Bir Bahadur Chaudhary |
|*Appeared on*| *2022-12-26*|
|*Comments*| *8 papers, 17 figures, Presented and Published at Proceedings of the XVII Vibration Engineering & Technology of Machinery Conference(VETOMAC),Dec. 15-17, 2022, DMAE, IOE, Pulchowk, Nepal*|
|**Abstract**| An increase in satellite application has skyrocketed the number of satellites, especially in the low earth orbit (LEO). The major concern today is that these satellites become debris after the end of life, negatively affecting the space environment. As per the International Guidelines of the European Space Agency, it is mandatory to deorbit the satellite within 25 years of its end of life. This paper is aimed to design the solid chemical propellant thruster to deorbit the StudSat II from its original orbit to the lower orbit. StudSat II carries the heritage of StudSat I, successfully launched on 12th July 2010 AD, and is the first Pico Satellite in India by the undergraduate students of seven engineering colleges. This paper explains how a solid monopropellant thruster could be used to deorbit the satellite after the end of life with the least difficulty compared to other active and passive methods of deorbiting. The deorbiting mechanism consists of a solid propellant, Convergent Divergent nozzle, ignition system, and electronic actuators. The components of the thruster were designed in the CATIA V5, and the combustion studies and flow analysis were done in ANSYS. The concept of Hohmann transfer was used to deorbit the satellite, and STK was used to simulate it. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.11991.md
    + _build/html/tmp_2212.11991/./bestfit_lz_percent.png
    + _build/html/tmp_2212.11991/./zvz_redblue_r.png
    + _build/html/tmp_2212.11991/./fit_trends1_residuals_v2.png
exported in  _build/html/2212.12039.md
    + _build/html/tmp_2212.12039/./ngc1365_full_nov_23_trim.png
    + _build/html/tmp_2212.12039/./ngc1365_region_2_nov_22_2022_trim.png
    + _build/html/tmp_2212.12039/./ngc1365_7plot_nov_23.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\rhalf}{\ensuremath{R_h}}$
$\newcommand{\rff}{\ensuremath{R_\mathrm{eff}}}$
$\newcommand{\rifu}{\ensuremath{R_\mathrm{IFU}}}$
$\newcommand{\Amax}{\ensuremath{A_\mathrm{2max}}}$
$\newcommand{\omegaP}{\ensuremath{\Omega_P}}$
$\newcommand{\feh}{\ensuremath{\mathrm{[Fe/H]}}}$
$\newcommand{\half}{\tfrac{1}{2}}$
$\newcommand{\amp}{\ensuremath{A}}$
$\newcommand{\Comment}[2]{ [{\color{red}\sc #1 :} {{\color{orange} \it #2}}]}$
$\newcommand{\com}[1]{ {\textcolor{cyan}{ #1}}}$
$\newcommand{\update}[1]{ {\textbf{\textcolor{mybrown}{ #1}}}}$
$\newcommand{\updateap}[1]{ {\textbf{\textcolor{cyan}{ #1}}}}$
$\newcommand{\updatescott}[1]{ {\textbf{\textcolor{magenta}{ #1}}}}$
$\newcommand{\apn}[1]{ {\textcolor{magenta}{ #1}}}$
$\newcommand{\jo}[1]{ \textbf{{\textcolor{myblue}{Jo asks/comments: #1}}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\rhalf$}{$\ensuremath${R_h}}$
$\newcommand{$\rff$}{$\ensuremath${R_\mathrm{eff}}}$
$\newcommand{$\rifu$}{$\ensuremath${R_\mathrm{IFU}}}$
$\newcommand{$\Amax$}{$\ensuremath${A_\mathrm{2max}}}$
$\newcommand{$\omegaP$}{$\ensuremath${\Omega_P}}$
$\newcommand{$\feh$}{$\ensuremath${\mathrm{[Fe/H]}}}$
$\newcommand{$\half$}{\tfrac{1}{2}}$
$\newcommand{$\amp$}{$\ensuremath${A}}$
$\newcommand{$\Comment$}[2]{ [{\color{red}\sc #1 :} {{\color{orange} \it #2}}]}$
$\newcommand{$\com$}[1]{ {\textcolor{cyan}{ #1}}}$
$\newcommand{$\update$}[1]{ {\textbf{\textcolor{mybrown}{ #1}}}}$
$\newcommand{$\update$ap}[1]{ {\textbf{\textcolor{cyan}{ #1}}}}$
$\newcommand{$\update$scott}[1]{ {\textbf{\textcolor{magenta}{ #1}}}}$
$\newcommand{$\apn$}[1]{ {\textcolor{magenta}{ #1}}}$
$\newcommand{$\jo$}[1]{ \textbf{{\textcolor{myblue}{Jo asks/comments: #1}}}}$</div>



<div id="title">

#  Vertical motion in the Galactic disc: unwinding the Snail

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.11991-b31b1b.svg)](https://arxiv.org/abs/2212.11991)<mark>Appeared on: 2022-12-26</mark> - _9 pages, 8 figures. Submitted to MNRAS (constructive comments are welcome)_

</div>
<div id="authors">

<mark><mark>Neige Frankel</mark></mark>, Jo Bovy, Scott Tremaine, David W. Hogg

</div>
<div id="abstract">

**Abstract:** \noindentThe distribution of stars in the Milky Way disc shows a spiral structure--the Snail--in the space of velocity and position normal to the Galactic mid-plane. The Snail appears as straight lines in the vertical frequency--vertical phase plane when effects from sample selection are removed. Their slope has the dimension of inverse time, with the simplest interpretation being the inverse age of the Snail. Here, we devise and fit a simple model in which the spiral starts as a lopsided perturbation from steady state, that winds up into the present-day morphology. The winding occurs because the vertical frequency decreases with vertical action. We use data from stars in\textsl{Gaia}EDR3 that have measured radial velocities, pruned by simple distance and photometric selection functions. We divide the data into boxels of dynamical invariants (radial action, angular momentum); our model fits the data well in many of the boxels. The model parameters have physical interpretations: one,$\amp$, is a perturbation amplitude, and one,$t$, is interpretable in the simplest models as the time since the event that caused the Snail. We find trends relating the strength and age to angular momentum: (i) the amplitude$\amp$is small at low angular momentum ($<1 600\mbox{ kpc km s}^{-1}$or guiding-centre radius$< 7.3 $kpc), and over a factor of three larger, with strong variations, in the outer disc; (ii) there is no single well-defined perturbation time, with$t$varying between 0.2 and 0.6 Gyr. Residuals between the data and the model display systematic trends, implying that the data call for more complex models.

</div>

<div id="div_fig1">

<img src="tmp_2212.11991/./bestfit_lz_percent.png" alt="Fig1" width="100%"/>

**Figure 1. -** Maximum-likelihood estimates of two of the parameters in Eq. (\ref{eq:logl}): the strength of the perturbation, $\amp$(top panel) and the time of the perturbation, $t$(bottom). Each solid line corresponds to the median of many different fitting experiments, as specified in the text, in which we varied the initial guess for the optimizer, the binning in $J_\varphi$ and the random subset bootstrapped from the data. The shaded areas enclose the 25$^\mathrm{th}$ and the 75$^\mathrm{th}$ percentiles, as determined from these experiments. The $\odot$ symbol marks the angular momentum of a circular orbit at the solar radius. (*fig:best_fit_trends*)

</div>
<div id="div_fig2">

<img src="tmp_2212.11991/./zvz_redblue_r.png" alt="Fig3" width="100%"/>

**Figure 3. -** Fractional density contrast in the $z$--$v_z$ plane, revealing the \textsl{Gaia} phase-space Snail. The vertical band near $z=0$ is due to dust extinction. The details of how this figure was constructed are given in \S\ref{section:data_selection}. (*fig:snail*)

</div>
<div id="div_fig3">

<img src="tmp_2212.11991/./fit_trends1_residuals_v2.png" alt="Fig6" width="100%"/>

**Figure 6. -** A row-normalized histogram of the number of stars as a function of angle $\theta_z$ at given vertical frequency $\Omega_z$ for 12 bins in angular momentum $J_\varphi$($\mathrm{kpc km s^{-1}}$), i.e., $p(\theta_z|\Omega_z)$. The lines of panels show the data (top), the best fit from the model of Eq. (\ref{eq:logl}) (middle), and the residuals between the data and this model (bottom). (*fig:best_fit_trends_res*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\kms}{ km s^{-1}}$
$\newcommand{\bibtex}{\textsc{Bib}\!\TeX}$
$\newcommand$
$\newcommand{\ngal}{ 17 }$
$\newcommand{\UWyoming}{\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}}$
$\newcommand{\STScI}{\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\UAntof}{\affiliation{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{\NOIRLab}{\affiliation{Gemini Observatory/NSF NOIRLab, 950 N. Cherry Avenue, Tucson, AZ 85719, USA}}$
$\newcommand{\UToledo}{\affiliation{Ritter Astrophysical Research Center, University of Toledo, Toledo, OH 43606, USA}}$
$\newcommand{\JHU}{\affiliation{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218 USA}}$
$\newcommand{\Caltech}{\affiliation{TAPIR, California Institute of Technology, Pasadena, CA 91125 USA}}$
$\newcommand{\OSU}{\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave., Columbus, OH 43210, USA}}$
$\newcommand{\MPIA}{\affiliation{Max Planck Institut f\"ur Astronomie, K\"onigstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{\ESO}{\affiliation{European Southern Observatory,  Karl-Schwarzschild Str. 2, 85748 Garching bei Muenchen, Germany}}$
$\newcommand{\UAlberta}{\affiliation{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}}$
$\newcommand{\UCSD}{\affiliation{Center for Astrophysics \& Space Sciences, Department of Physics,  University of California San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}}$
$\newcommand{\Belgium}{\affiliation{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}}$
$\newcommand{\UHeidelberg}{\affiliation{Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstr. 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{\Bonn}{\affiliation{Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121, Bonn, Germany}}$
$\newcommand{\ANU}{\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}}$
$\newcommand{\AthreeD}{\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}}$
$\newcommand{\nrao}{\affiliation{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903, USA}}$
$\newcommand{\ITA}{\affiliation{Institut f\"{u}r Theoretische Astrophysik, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg,\Albert-Ueberle-Strasse 2, 69120 Heidelberg, Germany}}$
$\newcommand{\COOL}{\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}}$
$\newcommand{\IWR}{\affiliation{Universit\"{a}t Heidelberg, Interdisziplin\"{a}res Zentrum f\"{u}r Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}}$
$\newcommand{\MPE}{\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}}$
$\newcommand{\ICRAR}{\affiliation{International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}}$
$\newcommand{\OCADU}{\affiliation{OCAD University, Toronto, Ontario, M5T 1W1, Canada}}$
$\newcommand{\UArizona}{\affiliation{Steward Observatory, University of Arizona, 933 N Cherry Ave,Tucson, AZ 85721, USA}}$
$\newcommand{\Riverside}{\affiliation{Department of Physics and Astronomy, University of California, Riverside, CA, 92521 USA}}$
$\newcommand{\Connecticut}{\affiliation{University of Connecticut, Department of Physics, 196A  Auditorium Road, Unit 3046, Storrs, CT, 06269}}$
$\newcommand{\ARI}{\affiliation{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}}$
$\newcommand{\LaPlata}{\affiliation{Instituto de Astrofisica de La Plata, CONICET–UNLP,$
$ Paseo del Bosque S/N, B1900FWA La Plata, Argentina }}$
$\newcommand{\sorbonne}{\affiliation{Sorbonne {Universit\'e}, LERMA, Observatoire de Paris, PSL university, CNRS, F-75014, Paris, France}}$
$\newcommand{\CITA}{\affiliation{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}}$
$\newcommand{\MCMASTER}{\affiliation{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}}$
$\newcommand{\lea}{\mathrel{<\kern-1.0em\lower0.9ex\hbox{\sim}}}$
$\newcommand{\gea}{\mathrel{>\kern-1.0em\lower0.9ex\hbox{\sim}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\kms$}{ km s^{-1}}$
$\newcommand{$\bibtex$}{$\textsc${Bib}\!\TeX}$
$\newcommand$
$\newcommand{$\ngal$}{ 17 }$
$\newcommand{$\UWyoming$}{\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}}$
$\newcommand{$\STScI$}{\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{$\UAntof$}{\affiliation{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{$\NOIRLab$}{\affiliation{Gemini Observatory/NSF NOIRLab, 950 N. Cherry Avenue, Tucson, AZ 85719, USA}}$
$\newcommand{$\UToledo$}{\affiliation{Ritter Astrophysical Research Center, University of Toledo, Toledo, OH 43606, USA}}$
$\newcommand{$\JHU$}{\affiliation{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218 USA}}$
$\newcommand{$\Caltech$}{\affiliation{TAPIR, California Institute of Technology, Pasadena, CA 91125 USA}}$
$\newcommand{$\OSU$}{\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave., Columbus, OH 43210, USA}}$
$\newcommand{$\MPIA$}{\affiliation{Max Planck Institut f\"ur Astronomie, K\"onigstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{$\ESO$}{\affiliation{European Southern Observatory,  Karl-Schwarzschild Str. 2, 85748 Garching bei Muenchen, Germany}}$
$\newcommand{$\UAlberta$}{\affiliation{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}}$
$\newcommand{$\UCSD$}{\affiliation{Center for Astrophysics \& Space Sciences, Department of Physics,  University of California San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}}$
$\newcommand{$\Belgium$}{\affiliation{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}}$
$\newcommand{$\UHeidelberg$}{\affiliation{Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstr. 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{$\Bonn$}{\affiliation{Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121, Bonn, Germany}}$
$\newcommand{$\ANU$}{\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}}$
$\newcommand{$\AthreeD$}{\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}}$
$\newcommand{$\nrao$}{\affiliation{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903, USA}}$
$\newcommand{$\ITA$}{\affiliation{Institut f\"{u}r Theoretische Astrophysik, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg,\Albert-Ueberle-Strasse 2, 69120 Heidelberg, Germany}}$
$\newcommand{$\COOL$}{\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}}$
$\newcommand{$\IWR$}{\affiliation{Universit\"{a}t Heidelberg, Interdisziplin\"{a}res Zentrum f\"{u}r Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}}$
$\newcommand{$\MPE$}{\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}}$
$\newcommand{$\ICRAR$}{\affiliation{International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}}$
$\newcommand{$\OCADU$}{\affiliation{OCAD University, Toronto, Ontario, M5T 1W1, Canada}}$
$\newcommand{$\UArizona$}{\affiliation{Steward Observatory, University of Arizona, 933 N Cherry Ave,Tucson, AZ 85721, USA}}$
$\newcommand{$\Riverside$}{\affiliation{Department of Physics and Astronomy, University of California, Riverside, CA, 92521 USA}}$
$\newcommand{$\Connecticut$}{\affiliation{University of Connecticut, Department of Physics, 196A  Auditorium Road, Unit 3046, Storrs, CT, 06269}}$
$\newcommand{$\ARI$}{\affiliation{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}}$
$\newcommand{\LaPlata}{\affiliation{Instituto de Astrofisica de La Plata, CONICET–UNLP,$
$ Paseo del Bosque S/N, B1900FWA La Plata, Argentina }}$
$\newcommand{$\sorbonne$}{\affiliation{Sorbonne {Universit\'e}, LERMA, Observatoire de Paris, PSL university, CNRS, F-75014, Paris, France}}$
$\newcommand{$\CITA$}{\affiliation{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}}$
$\newcommand{$\MCMASTER$}{\affiliation{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}}$
$\newcommand{$\lea$}{\mathrel{<\kern-1.0em\lower0.9ex\hbox{\sim}}}$
$\newcommand{$\gea$}{\mathrel{>\kern-1.0em\lower0.9ex\hbox{\sim}}}$</div>



<div id="title">

# PHANGS-JWST First Results: Massive Young Star Clusters and New Insights from JWST Observations of NGC 1365

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.12039-b31b1b.svg)](https://arxiv.org/abs/2212.12039)<mark>Appeared on: 2022-12-26</mark> - _17 pages, 9 figures, accepted for publications as part of PHANGS-JWST ApJL Focus Issue_

</div>
<div id="authors">

Bradley C. Whitmore, et al. -- incl., <mark><mark>Janice C. Lee</mark></mark>, <mark><mark>Kiana F. Henny</mark></mark>, <mark><mark>Stephen Hannon</mark></mark>, <mark><mark>Jonathan D. Henshaw</mark></mark>, <mark><mark>Eva Schinnerer</mark></mark>

</div>
<div id="abstract">

**Abstract:** A primary new capability of JWST is the ability to penetrate the dust in star forming galaxies to identify and study the properties ofyoung star clusters that remain embedded in dust and gas.In this paper we combine new infrared images taken with JWST with our optical HST images of the star-bursting barred (Seyfert2) spiral galaxy NGC 1365.We find that this galaxy has the richest population of massive young clusters of any known galaxy within 30 Mpc, with$\sim$30 star clusters that are more massive than$10^{6} M_{\odot}$and younger than 10 Myr. Sixteen of these clusters are newly discovered from our JWST observations.  An examination of the optical images reveals that 4 of 30 ($\sim 13$\%)  are so deeply embedded that they cannot be seen in the Hubble I band ($A_V $\gea$ 10$mag), and that 11 of 30 ($\sim37$\%) are missing in the HST B  band,so age and mass estimates from optical measurements alone are challenging.  These numbers suggest that massive clusters in NGC 1365 remain completely obscured in the visible for$\sim1.3\pm0.7$Myr, and are either completely or partially obscured for$\sim3.7\pm1.1$Myr.We also use the JWST observations to gain new insights intothe triggering of star cluster formation by the collision of gas and dust streamers with gas and dust in the bar.  The JWST images reveal previously unknown structures (e.g., bridges and overshoot regions from stars that form in the bar) that help us better understand the orbital dynamics of barred galaxies and associated star-forming rings.Finally, we note that the excellent spatial  resolution of the NIRCAM F200W filter provides a  better way to separate barely resolved compact clusters from individual stars based on their sizes.

</div>

<div id="div_fig1">

<img src="tmp_2212.12039/./ngc1365_full_nov_23_trim.png" alt="Fig1" width="100%"/>

**Figure 1. -** 
An optical IVB  color image taken by Hubble of NGC 1365 is shown in the upper panel,  and a JWST infrared (F770W) plus Hubble (V and B) image is shown in the bottom panel. The full field of view covers an area of roughly 80$"$$\times$ 160$"$(7.5 kpc $\times$ 15.0 kpc). The fields-of-view for regions discussed in the text are shown, as well as the motion along the bar and along two of the  streamers. We note that the extensive star formation in the spiral arm and the end of the Eastern bar on the left edge of the image is outside the F770W field of view. (*fig:full_galaxy*)

</div>
<div id="div_fig2">

<img src="tmp_2212.12039/./ngc1365_region_2_nov_22_2022_trim.png" alt="Fig5" width="100%"/>

**Figure 5. -**  JWST (F770W and F300M) and Hubble (B band) images of Region \# 2, just up the NE bar from the nucleus, as shown in Figure \ref{fig:full_galaxy}. The different contrast levels highlight key features, including two young 'bubbles' resulting from star-formation that may have been triggered by a intra-bar streamer colliding with gas and dust in the bar itself, as shown by the arrows. Another feature, which is only visible in the JWST bands due to the heavy dust in the area, is the "NE Bridge", as discussed in the text.  (*fig:reg_2*)

</div>
<div id="div_fig3">

<img src="tmp_2212.12039/./ngc1365_7plot_nov_23.png" alt="Fig7" width="100%"/>

**Figure 7. -** 
Region 1 in NGC 1365.
The figure shows an IVB image from Hubble in the upper left, and three images with different JWST bands  (F335M, F700W, or F2100) in red, F200W  (NIR continuum) in green, and I or V from Hubble in blue in the other panels.
This region is just north of the nucleus, and contains the three strongest radio continuum sources in this galaxy, which appear to be young
star clusters with masses of roughly log Mass in the range 6 to 7,
(referred to as M4, M5, M6 by \citealt{sandqvist95}, \citealt{galliano08}, \citealt{galliano2012}).
Several thin dark filaments are shown along a ridge line containing some of the most recent star cluster formation in the F335M image. The holes in M4, M5, and M6 in both the F2100W and F770W images show where the data is saturated.
The largely unobscured region to the upper right shows  an "overshoot region" outside the inner part of the galaxy
(see \citealt{sormani20}), with most of the clusters having estimated ages in the range between 10 to 20  Myr. The two bottom left panels show the excellent resolution of the JWST F200W image, and its ability to cut through the dust better than the I band Hubble image, which is shown to the right.  The bottom right panel shows the CO (2-1) line intensity emission map (Schinnerer et al. 2022, this volume) for Region 1. The small red cross in the top four panels is an artifact of the Hubble Legacy Archive which we used to make the figures, and should be ignored.
Linear diagonal artifacts from the saturated nucleus are visible at the bottom edge of the three color panels using JWST filters.
 (*fig:reg_1*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

148  publications in the last 7 days.
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers